In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Model, GPT2Config, PreTrainedModel
from datasets import load_dataset, Dataset, DatasetDict

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

import math

In [2]:
class GPT2Trader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        # use levine 2020 layer numbers
        n_layer = round((math.log(config.n_embd) - 5.039) / 5.55e-2)
        n_layer = max(1, n_layer)
        print(f'Using {n_layer} layers')
        config.n_layer = n_layer
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.norm = nn.LayerNorm(config.n_embd)
        self.gpt = GPT2Model(config)
        self.trade = nn.Linear(config.n_embd, 60, bias = False)
        
        self.trade_sign = nn.Parameter(torch.Tensor([1, -1]), requires_grad = False)
        
    def forward(self, ohlcv, future):
        embed = self.norm(self.embed(ohlcv))
        hidden = self.gpt(inputs_embeds = embed).last_hidden_state
        
        soft_trade = self.trade(hidden).reshape(-1)
        
        # sharpe information
        soft_trade = torch.tanh(soft_trade)
        soft_profit = soft_trade * future.reshape(-1)
        
        # the exp is so that loss is purely positive and minimizes toward 0 (also losses have more loss than profit)
        loss_ppl = torch.exp(-soft_profit).mean()
        
        # penalty for big trades (to stop trading from happening with no profit)
        trade_penalty = soft_trade.abs().mean()
        
        loss = loss_ppl + .05 * trade_penalty # .05 since the .05 * std ~ commission size
        
        trades = soft_trade.abs().sum()
        print(f'sharpe {soft_profit.mean() / soft_profit.std():.4}, t-value {(soft_profit.mean() * trades**.5) / soft_profit.std():.4} '
              f'trade % {trades * 100 / len(soft_profit):,.4}, big trade % {(soft_trade.abs() > .5).sum() * 100 / len(soft_profit):.4}, loss {loss_ppl.item():.4}')
        
        return {"loss": loss}

In [3]:
class TraderTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training :class:`~torch.utils.data.DataLoader`.

        Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
        to distributed training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            shuffle=False, # IMPORTANT TO STOP OVERFITTING
#             sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

In [4]:
eurusd = Dataset.from_parquet('data/EURUSD_day.pq')

# make splits
split = eurusd.train_test_split(.1, shuffle = False)
valid_test = split['test'].train_test_split(.5, shuffle = False)
eurusd = DatasetDict({
    'train': split['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

Using custom data configuration default-524475b7137ed866
Found cached dataset parquet (C:/Users/micha/.cache/huggingface/datasets/parquet/default-524475b7137ed866/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 200,
    eval_steps = 200,
    report_to = "none",
    learning_rate = 5e-4,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    max_grad_norm = 1,
)

In [6]:
config = GPT2Config(
    n_embd = 320, n_head = 5, vocab_size = 0, n_positions = 2000,
    resid_pdrop = .1, embd_pdrop = .1, attn_pdrop = .1,
#     resid_pdrop = .0, embd_pdrop = .0, attn_pdrop = .0,
    scale_attn_by_inverse_layer_idx = True, use_cache = False
)

In [7]:
model = GPT2Trader(config).cuda()
trainer = TraderTrainer(
    model = model,
    args = training_args,
    train_dataset = eurusd['train'],
    eval_dataset = eurusd['validation']
)

Using 13 layers


In [8]:
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3098
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3098
Could not estimate the number of tokens of the input, floating-point operations will not be computed


sharpe 0.009729, t-value 1.747 trade % 37.31, big trade % 31.04, loss 1.86e+12


Step,Training Loss,Validation Loss
200,55319559950827.523438,1.000676
400,11.988700,1.001720
600,1.354300,1.024451
800,1.051000,0.999747
1000,1.000300,0.999269
1200,0.999500,0.999727
1400,1.000300,0.999136
1600,0.999500,0.999085
1800,1.000500,0.998532
2000,0.999100,0.997905


sharpe 0.007773, t-value 1.384 trade % 36.7, big trade % 30.09, loss 1.631
sharpe 0.0101, t-value 1.777 trade % 35.84, big trade % 28.63, loss 172.3
sharpe 0.005823, t-value 1.022 trade % 35.66, big trade % 28.47, loss 3.555
sharpe 0.005891, t-value 1.066 trade % 37.93, big trade % 32.25, loss 6.689e+11
sharpe 0.01624, t-value 2.815 trade % 34.76, big trade % 26.58, loss 1.394
sharpe 0.003126, t-value 0.5439 trade % 35.04, big trade % 27.45, loss 1.791e+05
sharpe 0.01285, t-value 2.177 trade % 33.2, big trade % 24.49, loss 1.295
sharpe 0.02582, t-value 4.311 trade % 32.28, big trade % 22.51, loss 1.583
sharpe 0.01642, t-value 2.698 trade % 31.26, big trade % 21.43, loss 1.605
sharpe 0.02375, t-value 3.782 trade % 29.34, big trade % 17.58, loss 77.4
sharpe 0.01212, t-value 1.981 trade % 30.9, big trade % 20.91, loss 18.08
sharpe 0.0428, t-value 7.009 trade % 31.04, big trade % 20.74, loss 2.786
sharpe 0.08478, t-value 13.66 trade % 30.04, big trade % 18.81, loss 1.146
sharpe 0.03782, t-

***** Running Evaluation *****
  Num examples = 172
  Batch size = 1


...

sharpe 0.142, t-value 9.534 trade % 5.218, big trade % 0.0, loss 0.9964
sharpe 0.05013, t-value 3.475 trade % 5.561, big trade % 0.0, loss 0.9992
sharpe 0.1159, t-value 5.824 trade % 2.925, big trade % 0.0, loss 0.9957
sharpe 0.08138, t-value 6.576 trade % 7.558, big trade % 0.0, loss 0.996
sharpe 0.157, t-value 11.09 trade % 5.773, big trade % 0.0, loss 0.9965
sharpe 0.1101, t-value 6.965 trade % 4.636, big trade % 0.0, loss 0.998
sharpe 0.1955, t-value 11.26 trade % 3.839, big trade % 0.0, loss 0.9898
sharpe 0.1365, t-value 9.685 trade % 5.824, big trade % 0.0, loss 0.9962
sharpe 0.1143, t-value 6.158 trade % 3.361, big trade % 0.0, loss 0.9924
sharpe 0.1479, t-value 10.73 trade % 6.093, big trade % 0.0, loss 0.9934
sharpe 0.1139, t-value 6.713 trade % 4.019, big trade % 0.0, loss 0.9927
sharpe 0.2162, t-value 14.75 trade % 5.39, big trade % 0.0, loss 0.9942
sharpe 0.1097, t-value 8.202 trade % 6.467, big trade % 0.0, loss 0.9934
sharpe 0.1402, t-value 10.24 trade % 6.178, big trade 

Saving model checkpoint to ./results\checkpoint-3000
Configuration saved in ./results\checkpoint-3000\config.json
Model weights saved in ./results\checkpoint-3000\pytorch_model.bin


sharpe 0.1196, t-value 10.46 trade % 8.865, big trade % 0.0, loss 0.9948
sharpe 0.1547, t-value 12.32 trade % 7.344, big trade % 0.0, loss 0.996
sharpe 0.05948, t-value 4.497 trade % 6.616, big trade % 0.0, loss 0.9987
sharpe 0.07496, t-value 6.221 trade % 7.97, big trade % 0.0, loss 0.9971
sharpe 0.1804, t-value 17.09 trade % 10.39, big trade % 0.0, loss 0.9877
sharpe 0.02308, t-value 1.921 trade % 8.019, big trade % 0.0, loss 1.001
sharpe 0.1388, t-value 11.81 trade % 8.385, big trade % 0.0, loss 0.9934
sharpe 0.1457, t-value 14.33 trade % 11.2, big trade % 0.0, loss 0.9956
sharpe 0.1274, t-value 11.99 trade % 10.25, big trade % 0.0, loss 0.993
sharpe 0.04808, t-value 2.83 trade % 4.011, big trade % 0.0, loss 0.9987
sharpe 0.05727, t-value 4.798 trade % 8.126, big trade % 0.0, loss 0.9982
sharpe 0.1214, t-value 10.08 trade % 7.975, big trade % 0.0, loss 0.9947
sharpe 0.08571, t-value 7.563 trade % 9.011, big trade % 0.0, loss 0.9963
sharpe 0.05949, t-value 3.875 trade % 4.91, big tra



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3098, training_loss=3571307937434.048, metrics={'train_runtime': 1079.3329, 'train_samples_per_second': 2.87, 'train_steps_per_second': 2.87, 'total_flos': 0.0, 'train_loss': 3571307937434.048, 'epoch': 1.0})

In [9]:
del trainer
del model
torch.cuda.empty_cache()

# Appendix

## quick timing check

In [ ]:
model = GPT2Trader(config).cuda()

In [72]:
%%timeit
fake_data = torch.randn(4, 391, 256)
fake_data = fake_data.cuda()
model(fake_data)
cpu = fake_data.cpu()

28.2 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
model = GPT2Trader(config).cpu()

Using 9 layers


In [74]:
%%timeit
fake_data = torch.randn(4, 391, 256)
model(fake_data)

748 ms ± 82.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
